In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from Global_Parameter import GlobalParams
from Green_Function import GKTH_Greens
from H_eig import GKTH_find_spectrum
from k_space_flipping import GKTH_flipflip
from Layer import Layer
from self_consistency_delta import GKTH_self_consistency_1S

In [ ]:
p = GlobalParams()
Nb = Layer()
layers = [Nb]

ts_list = []

# Band energy spectrums

In [ ]:
max_val_kx = np.max(np.abs(p.k1))
kx = np.linspace(-max_val_kx, max_val_kx, p.nkpoints * 2)
ky = np.linspace(-max_val_kx, max_val_kx, p.nkpoints * 2)
kx, ky = np.meshgrid(kx, ky)

# energy is 3d array, nkpoints x nkpoints x (4*nlayers)
# each k-point has its only eigenvalues list
energy = GKTH_find_spectrum(p, layers)

bands = []
for i in range(4 * len(layers)):
    energy_band = energy[:, :, i]
    new_band = GKTH_flipflip(energy_band)
    bands.append(new_band)

In [ ]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

for i, new_band in enumerate(bands):
    # Skip bands with NaNs or infinities
    if np.isnan(new_band).any() or np.isinf(new_band).any():
        print(f"Skipping band {i} due to NaN or infinity values.")
        continue

    surf = ax.plot_surface(kx, ky, new_band, facecolor=f"C{i%10}", alpha=0.5)

ax.set_xlabel("$k_x$")
ax.set_ylabel("$k_y$")
ax.set_zlabel("Energy (eV)")
ax.set_title("Energy Spectrum")

In [ ]:
%matplotlib inline
plt.contourf(kx, ky, new_band, 100)
plt.gca().set_aspect("equal")
ax.set_xlabel("$k_x$")
ax.set_ylabel("$k_y$")
plt.colorbar()

# Delta

In [ ]:
Delta, layers, residual_history = GKTH_self_consistency_1S(p, layers)

In [ ]:
%matplotlib inline
matsubara_freqs, ksums, F_kresolved_final = GKTH_Greens(p, layers, verbose = True)

In [ ]:
plt.pcolormesh(np.real(F_kresolved_final[0, ...]))
plt.colorbar()
plt.title("Real part of F_kresolved_final")
plt.show()

plt.scatter(matsubara_freqs, np.abs(ksums))
plt.xlim(0, 5200)
plt.title("Matsubara Frequencies vs |ksums|")
plt.xlabel("Matsubara Frequencies")
plt.ylabel("|ksums|")
plt.show()

# Dump

In [ ]:
# fig = plt.figure()
# for i in range(10):
#     ts = 0.9 + i/50
#     p.ts = np.zeros(100) + ts
#     delta_list = []
#     tNN_list = []
#     for j in range(10):
#         print("current iteration:", i, j)
#         ts_list.append(ts)
#         tNN = -1 - j/20
#         Nb.tNN = tNN
#         Nb.tNNN = Nb.tNN * 0.1
#         tNN_list.append(tNN)
#         delta,_, _ = GKTH_self_consistency_1S(p,layers)
#         delta_list.append(delta)
#     plt.plot(tNN_list, delta_list, label = f"{ts:.2f}")
    
# plt.xlabel("tNN (eV)")
# plt.ylabel("delta_best_fit (eV)")
# plt.legend()